In [1]:
# Lightning related imports

# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning fsspec --no-deps --target=$nb_path
# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning
!pip install torchtext==0.8.0 torch==1.7.0 torchvision==0.8 pytorch-lightning==1.2.2
!pip install neptune-client==0.5.1

# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning
# https://github.com/PyTorchLightning/pytorch-lightning/issues/6210

     |████████████████████████████████| 7.0MB 6.8MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 11.8MB 8.6MB/s 
     |████████████████████████████████| 819kB 40.4MB/s 
     |████████████████████████████████| 276kB 59.7MB/s 
     |████████████████████████████████| 112kB 51.7MB/s 
     |████████████████████████████████| 1.3MB 44.9MB/s 
     |████████████████████████████████| 143kB 58.0MB/s 
     |████████████████████████████████| 296kB 58.9MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=19159019e1971d50a60c472669f4e891faa21350263b7bd3cde4a356d8a08ba0
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
ERROR: pytorch-lightning 1.2.2 has requirement future>=0.17.1, but you'll have future 0.16.0 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
  

In [2]:
# !pip install torch==1.7.0 torchvision==0.8 torchtext==0.8
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric
!pip install ogb

     |████████████████████████████████| 2.8MB 7.0MB/s 
     |████████████████████████████████| 1.6MB 6.3MB/s 
     |████████████████████████████████| 194kB 7.2MB/s 
     |████████████████████████████████| 235kB 10.7MB/s 
     |████████████████████████████████| 2.2MB 14.1MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 61kB 4.5MB/s 
  Created wheel for outdated: filename=outdated-0.2.0-cp37-none-any.whl size=4962 sha256=cd0df5e83e4f4d5f9674d512cd72be6017b49c78c544f3cb1e44b17298dff2f1
  Stored in directory: /root/.cache/pip/wheels/fd/7c/ef/814f514d31197310872b5abf353feb8fef9d67ee658e1e7e39
  Created wheel for littleutils: filename=littleutils-0.2.2-cp37-none-any.whl size=7051 sha256=53ebcd13044b75ea2ec309228b4cebdf7a09058f8bc12982a233a1754f5dbfe7
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built outdated littleutils


In [62]:
%load_ext autoreload
%autoreload 2
import pytorch_lightning
print(pytorch_lightning.__version__)
from pytorch_lightning import Trainer
from pytorch_lightning.loggers.neptune import NeptuneLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

import json
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
# from scikitplot.metrics import plot_confusion_matrix
from google.colab import files
from lightning_model import LightningPAN


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1.2.2


In [63]:
# Testing if the lightning_model script gets refreshed correctly
LightningPAN.training_epoch_end("hello",'Hle')


hi?


TypeError: ignored

In [65]:

# loading params
with open('parameters.json') as json_file:
    parameters = json.load(json_file)
print(parameters)


datasetname = parameters["dataset_name"]
phi = parameters["phi"]
runs = parameters["runs"]
batch_size = parameters["batch_size"]
filter_size = parameters["maximum_path_size"]+1
learning_rate = parameters["learning_rate"]
weight_decay = parameters["weight_decay"]
pool_ratio = parameters["pool_ratio"] # fraction of nodes to use for pooled feature array
pan_pool_weight = parameters['pan_pool_weight'] # importance weight on global matrix X
criterion_pos_weight = parameters['criterion_pos_weight'] # Over-weightage on positive labels
nhid = parameters["nhid"]
epochs = parameters['epochs']

LearningRateMonitor_Params = {'logging_interval': 'epoch'}
lr_logger = LearningRateMonitor(**LearningRateMonitor_Params)

checkpoint_callback = ModelCheckpoint(
    monitor='validation_loss',
    save_top_k=5,
    mode='min',
    dirpath='/content/models/',
    filename='PAN-{epoch:02d}-{validation_loss:.2f}-{rocauc_eval:.2f}',
    period=1
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

model = LightningPAN(9, 1, nhid=nhid, 
                     ratio=pool_ratio, pan_pool_weight=pan_pool_weight,
                     criterion_pos_weight=criterion_pos_weight,
                     filter_size=filter_size)
neptune_logger = NeptuneLogger(
                api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMDk5YjVmYzYtNTU0My00MzhkLWJiYTAtMGM4ZGVhZmEyMTZiIn0=",
                project_name='hvergnes/PAN',
                close_after_fit=False,
                params=parameters, # your hyperparameters, immutable
                tags=['PAN', 'parameters as described in paper'],  # tags
                upload_source_files=["parameters.json", "lightning_model.py"]
                )

trainer = Trainer(
    max_epochs=epochs,
    logger=neptune_logger,
    callbacks=[checkpoint_callback],
    gpus=1,
    # fast_dev_run=True,
)

model.cuda()
trainer.fit(model)

# Save top 5 models based on validation loss
!zip -r /content/models_checkpoint.zip /content/models
files.download("/content/models_checkpoint.zip")

# Save model
# print(os.listdir())
# torch.save(model.state_dict(), 'model.pt')
# print(os.listdir())
# files.download('model.pt') 

trainer.test(model)

NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: None, using: 0 TPU cores


{'dataset_name': 'ogbg-molhiv', 'phi': 0.3, 'runs': 1, 'num_workers': 0, 'batch_size': 128, 'maximum_path_size': 3, 'learning_rate': 0.001, 'weight_decay': 0.0005, 'pool_ratio': 0.5, 'pan_pool_weight': 0.5, 'criterion_pos_weight': 25, 'nhid': 512, 'epochs': 150}
Device: cuda
https://ui.neptune.ai/hvergnes/PAN/e/PAN-158



   | Name         | Type              | Params
----------------------------------------------------
0  | criterion    | BCEWithLogitsLoss | 0     
1  | atom_encoder | AtomEncoder       | 5.5 K 
2  | conv1        | PANConv           | 16.9 K
3  | norm1        | Norm              | 1.5 K 
4  | pool1        | PANPooling        | 518   
5  | drop1        | PANDropout        | 0     
6  | conv2        | PANConv           | 131 K 
7  | norm2        | Norm              | 768   
8  | pool2        | PANPooling        | 261   
9  | drop2        | PANDropout        | 0     
10 | conv3        | PANConv           | 32.9 K
11 | norm3        | Norm              | 384   
12 | pool3        | PANPooling        | 133   
13 | lin1         | Linear            | 8.3 K 
14 | bn1          | BatchNorm1d       | 128   
15 | lin2         | Linear            | 2.1 K 
16 | bn2          | BatchNorm1d       | 64    
17 | lin3         | Linear            | 33    
----------------------------------------------------


hi?


hi?


...


  adding: content/models/ (stored 0%)
  adding: content/models/PAN-epoch=98-validation_loss=23.37-rocauc_eval=0.79.ckpt (deflated 6%)
  adding: content/models/PAN-epoch=110-validation_loss=24.26-rocauc_eval=0.80.ckpt (deflated 6%)
  adding: content/models/PAN-epoch=74-validation_loss=24.40-rocauc_eval=0.77.ckpt (deflated 6%)
  adding: content/models/PAN-epoch=92-validation_loss=24.39-rocauc_eval=0.79.ckpt (deflated 6%)
  adding: content/models/PAN-epoch=76-validation_loss=24.28-rocauc_eval=0.77.ckpt (deflated 6%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.7217115046640531, 'test_loss': 55.18900838494301}
--------------------------------------------------------------------------------


[{'rocauc_test': 0.7217115046640531, 'test_loss': 55.18900838494301}]

In [73]:
from os import listdir
from os.path import isfile, join
mypath = "/content/models/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['PAN-epoch=98-validation_loss=23.37-rocauc_eval=0.79.ckpt',
 'PAN-epoch=110-validation_loss=24.26-rocauc_eval=0.80.ckpt',
 'PAN-epoch=74-validation_loss=24.40-rocauc_eval=0.77.ckpt',
 'PAN-epoch=92-validation_loss=24.39-rocauc_eval=0.79.ckpt',
 'PAN-epoch=76-validation_loss=24.28-rocauc_eval=0.77.ckpt']

In [76]:
for ckpt_nm in onlyfiles:
  print()
  print(ckpt_nm)
  ckpt_mod = LightningPAN.load_from_checkpoint(checkpoint_path="/content//models/"+ckpt_nm)
  ckpt_mod.eval()
  print(trainer.test(ckpt_mod))


PAN-epoch=98-validation_loss=23.37-rocauc_eval=0.79.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.7316441028216072, 'test_loss': 37.502220183610916}
--------------------------------------------------------------------------------
[{'rocauc_test': 0.7316441028216072, 'test_loss': 37.502220183610916}]

PAN-epoch=110-validation_loss=24.26-rocauc_eval=0.80.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.7217385426524268, 'test_loss': 40.247160628437996}
--------------------------------------------------------------------------------
[{'rocauc_test': 0.7217385426524268, 'test_loss': 40.247160628437996}]

PAN-epoch=74-validation_loss=24.40-rocauc_eval=0.77.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.7288456710249329, 'test_loss': 34.48842149972916}
--------------------------------------------------------------------------------
[{'rocauc_test': 0.7288456710249329, 'test_loss': 34.48842149972916}]

PAN-epoch=92-validation_loss=24.39-rocauc_eval=0.79.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.742022827787327, 'test_loss': 34.99248963594437}
--------------------------------------------------------------------------------
[{'rocauc_test': 0.742022827787327, 'test_loss': 34.99248963594437}]

PAN-epoch=76-validation_loss=24.28-rocauc_eval=0.77.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.7534830722879933, 'test_loss': 33.27610766887665}
--------------------------------------------------------------------------------
[{'rocauc_test': 0.7534830722879933, 'test_loss': 33.27610766887665}]


In [ ]:
# Command to keep colab running (run on browser console (i.e. chrome, cmd+shift+c))
# function ClickConnect(){
#     console.log("Clicked on connect button"); 
#     document.querySelector("colab-connect-button").click()
# }
# setInterval(ClickConnect,60000)

# var startClickConnect = function startClickConnect(){
#     var clickConnect = function clickConnect(){
#         console.log("Connnect Clicked - Start");
#         document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
#         console.log("Connnect Clicked - End"); 
#     };

#     var intervalId = setInterval(clickConnect, 60000);

#     var stopClickConnectHandler = function stopClickConnect() {
#         console.log("Connnect Clicked Stopped - Start");
#         clearInterval(intervalId);
#         console.log("Connnect Clicked Stopped - End");
#     };

#     return stopClickConnectHandler;
# };

# var stopClickConnect = startClickConnect();

# stopClickConnect();


In [ ]:
# # Importing dataset to do some quick EDA
# from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
# # Load the dataset 
# dataset = PygGraphPropPredDataset(name='ogbg-molhiv')

# split_idx = dataset.get_idx_split()

# # Check task type
# print('Task type: {}'.format(dataset.task_type))

In [ ]:
# directed_g, undirected_g = 0, 0
# neg_graphs = []; pos_graphs = []
# num_edges_lst = [];
# num_nodes_lst = [];
# for i, curr_g in enumerate(dataset):
#   num_edges_lst.append(curr_g.num_edges)
#   num_nodes_lst.append(curr_g.num_nodes)
#   if curr_g.is_directed():
#     directed_g += 1
#   else:
#     undirected_g += 1
  
#   if curr_g.y.item() == 0:
#     neg_graphs.append(i)
#   elif curr_g.y.item() == 1:
#     pos_graphs.append(i)
#   else:
#     continue

In [ ]:
# print("Total number of graphs:", len(dataset))
# print("Number of classes:", dataset.num_classes)
# print("Dimension of node features:", dataset.num_node_features)
# print("Dimension of edge features:", dataset.num_edge_features)
# print("Number of positive labels:", dataset.data.y.sum().item())
# print("Number of negative labels:", dataset.data.y.shape[0] - dataset.data.y.sum().item())
# print("Min, Mean, Max # edges:", np.min(num_edges_lst), round(np.mean(num_edges_lst),2), np.max(num_edges_lst))
# print("Min, Mean, Max # nodes:", np.min(num_nodes_lst), round(np.mean(num_nodes_lst),2), np.max(num_nodes_lst))
# print("Number of directed graphs:", directed_g)
# print("Number of undirected graphs:", undirected_g)